In [1]:
import numpy as np
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow.keras import layers
from sklearn.metrics import f1_score

2023-05-30 14:10:53.142472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the dataset
data_path = '/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/'
dataset = np.load(data_path + 'PCA_dataset1.npy', allow_pickle=True)[()]

x_train = dataset['train_stars_pca']
x_val = dataset['validation_stars_pca']
x_test = dataset['test_stars_pca']
y_train = dataset['train_C']
y_val = dataset['validation_C']
y_test = dataset['test_C']
SED_test = dataset['test_SEDs']


In [3]:
# Hyperparameters
PCA_components = 24
model_learning_rate = 0.1
N_epochs = 50
N_committee = 48

In [4]:
def SEDlisttoC(SED_list):
    sed_array = np.array(SED_list)
    return sed_array*0.5 + 1.5

def CtoSEDarray(c_values, variance):
    sed_classes = ((c_values - 1.25) // 0.5).astype(int)
    sed_classes = np.where((c_values < 1.25) | (c_values > 7.75), 20, sed_classes)
    sed_classes = np.where((variance > 1.00), 20, sed_classes)
    return sed_classes

In [5]:
# Define the model architecture
def create_model():
    initializer = tf.keras.initializers.GlorotNormal(seed = 1)
    model = tf.keras.Sequential([
        layers.Dense(26, input_shape=[PCA_components], activation='sigmoid', kernel_initializer= initializer),
        layers.Dense(26, activation='sigmoid', kernel_initializer= initializer),
        layers.Dense(1, activation = 'linear', kernel_initializer= initializer)
    ])
    model.compile(
        loss = tf.keras.losses.MeanSquaredError(),
        optimizer = tf.keras.optimizers.Adam(learning_rate = model_learning_rate)
    )
    return model

# Train a committee of 48 neural networks and make predictions
committee = []
committee_predictions = []
for i in range(N_committee):
    model = create_model()
    model.fit(x_train, y_train, epochs= N_epochs, verbose=0, validation_data=(x_val,y_val))
    committee.append(model)
    committee_predictions.append(model.predict(x_test, verbose = 1).reshape(-1)) # Predict the scalar parameter C using the committee    

committee_predictions = np.array(committee_predictions)
y_pred = np.mean(committee_predictions, axis=0)
pred_variance = np.var(committee_predictions, axis=0)
SED_pred = CtoSEDarray(y_pred,pred_variance)
#add metrics


1/1 [==============================] - 0s 49ms/step


In [6]:
tf.math.confusion_matrix(SED_test, SED_pred) 

<tf.Tensor: shape=(12, 12), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]], dtype=int32)>

In [9]:
# Evaluate the performance of the committee

mse = np.mean((y_test - y_pred)**2)
print('MSE:', mse)
print(pred_variance)

f1 = f1_score(SED_test, SED_pred, average = None)
print('F1 score:', f1)
print(np.mean(f1[:13]))

for i in range(len(y_pred)):
    print(SED_test[i], y_pred[i], SED_pred[i])


MSE: 2.5850226317367286
[5.7364482e-12 5.6511831e-12 3.8558787e-12 7.5175421e-12 7.3517486e-12
 8.0575546e-12 7.0912165e-12 3.9837764e-12 4.7653734e-12 9.6633812e-12]
F1 score: [0.         0.         0.         0.18181818 0.         0.
 0.         0.        ]
0.022727272727272728
0 4.6000323 6
11 4.6000323 6
4 4.6000323 6
7 4.6000323 6
6 4.6000323 6
3 4.6000323 6
3 4.6000323 6
4 4.6000323 6
9 4.6000323 6
8 4.6000323 6
